# 2022-09-21 analysis

This study examines the dependence of the x-y-z hollowing at HZ04 on the initial correlations in the bunch. The initial bunch was decorrelated by randomly permuting the (x, x'), (y, y'), (z, dE) pairs.

In [ ]:
import sys
import os
from tqdm.notebook import tqdm
from tqdm.notebook import trange
import importlib
from pprint import pprint
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import proplot as pplt
from ipywidgets import widgets
from ipywidgets import interactive

sys.path.append('/Users/46h/Research/')
from btfsim.analysis.utils import load_bunch
from btfsim.analysis.utils import load_history
from beamphys import plotting as mplt
from beamphys import utils
from beamphys import dist

## Settings

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['figure.facecolor'] = 'white'
pplt.rc['savefig.dpi'] = 300
# pplt.rc['pdf.fonttype'] = 42

In [ ]:
folder = '/Users/46h/Dropbox (ORNL)/work/btf/btf-sim/2022-09-21_RFQbunch_MEBT1_decorrelated/'    
os.listdir(folder)

In [ ]:
fig_path = os.path.join(folder, 'figures')
if not os.path.isdir(fig_path):
    os.mkdir(fig_path)

In [ ]:
def save(figname):
    plt.savefig(os.path.join(fig_path, f'{figname}.png'))
    return

## Load data

In [ ]:
prefix = '220921152418-run_btf-0-HZ04'
nodes = ['init', 'QH01', 'QV02', 'QH03', 'QV04', 'HZ04']

In [ ]:
filenames = {'history': os.path.join(folder, f'{prefix}-history.dat')}
filenames['bunch'] = dict()
for node in nodes:
    if node == 'init' or node == 'HZ04':
        _filename = f'{prefix}-bunch-{node}.dat'
    else:
        _filename = f'{prefix}-bunch-MEBT:{node}.dat'
    filenames['bunch'][node] = os.path.join(folder, _filename)
pprint(filenames)

## History

In [ ]:
history = load_history(filenames['history'])
history.head()

In [ ]:
fig_kws = dict(figsize=(2.75, 2))
plot_kws = dict()

In [ ]:
fig, ax = pplt.subplots(**fig_kws)
for dim in ['x', 'y', 'z']:
    data = history[f'eps_{dim}'].values
    data /= data[0]
    ax.plot(history['s'], data, label=r'$\varepsilon_{}$'.format(dim), **plot_kws)
ax.legend(ncol=1, loc='upper left', ms=3)
ax.format(xlabel='Distance [m]', ylabel='Relative growth')
save(f'relative_rms_emittance_growth')

In [ ]:
fig, ax = pplt.subplots(**fig_kws)
for i, dim in enumerate(['x', 'y', 'z']):
    i *= 2
    col = f'sig_{i}{i}'
    data = np.sqrt(history[col].values)
    data *= 1e3  # convert [m] to [mm]
    ax.plot(history['s'], data, label=r'${}$'.format(dim), **plot_kws)
ax.legend(ncol=1, loc='upper left')
ax.format(xlabel='Distance [m]', ylabel='RMS beam size [mm]')
save(f'rms_beam_size')

In [ ]:
fig, axes = pplt.subplots(ncols=2, figsize=(5, 2), sharey=False)
for ax, param in zip(axes, ['beta', 'alpha']):
    for dim in ['x', 'y']:
        col = f'{param}_{dim}'
        xdata = history['s'].values
        ydata = history[col].values
        ax.plot(xdata, ydata, label=r'$\{}_{}$'.format(param, dim))
    ax.legend(ncol=1, loc='upper left', handlelength=1.5)
axes.format(xlabel='Distance [m]')
axes[0].format(ylabel='[m/rad]')
save(f'twiss_beta_alpha')

## Bunch

In [ ]:
dims = ["x", "x'", "y", "y'", "z", "dE"]
units = ["mm", "mrad", "mm", "mrad", "mm", "keV"]
labels = [f'{d} [{u}]' for d, u in zip(dims, units)]

### Interactive

In [ ]:
prof_kws=dict(lw=0.7, alpha=0.6, color='white', scale=0.09)

mplt.interactive_proj2d_discrete(
    load_bunch(filenames['bunch']['HZ04']),
    nbins=40,
    dims=dims,
    units=units,
    prof_kws=prof_kws,
)

### Sliced x-z distribution

At each node in the lattice, plot the x-z distribution for the following slices: (i) full projection, (ii) $y \approx 0$, (iii) $y \approx y' \approx 0$, (iv) $y \approx y' \approx x' \approx 0$. We normalize the coordinates to unit variance in each plot.

In [ ]:
bins = 40
for node in nodes:
    X = load_bunch(filenames['bunch'][node])
    Sigma = np.cov(X.T)
    X = X / np.sqrt(Sigma.diagonal())
    
    maxs = np.max(X, axis=0)
    xlim = (-maxs[0], maxs[0])
    ylim = (-maxs[4], maxs[4])
    edges = [np.histogram_bin_edges(X[:, i], bins, (-maxs[i], maxs[i])) 
             for i in range(X.shape[1])]
    slice_widths = [np.diff(e)[0] for e in edges]

    fig, axes = pplt.subplots(ncols=2, nrows=2,  figwidth=4.0)
    axes.format(xlim=xlim, ylim=ylim, xlabel='x [mm]', ylabel='z [mm]')
    for ax, islice in zip(axes, [None, (2,), (2, 3), (2, 3, 1)]):
        if islice is None:
            _X = X
        else:
            _slice_widths = [slice_widths[i] for i in islice]
            _X = dist.slice_box(X, axis=islice, limits=[(-w, w) for w in _slice_widths])
        _im, _edges = np.histogramdd(_X[:, (0, 4)], bins, (xlim, ylim))
        _centers = [utils.get_bin_centers(e) for e in _edges]
        mplt.plot_image(
            _im, 
            x=utils.get_bin_centers(_edges[0]), 
            y=utils.get_bin_centers(_edges[1]),
            ax=ax,
        )
        if islice is None:
            continue
        dims_n = [r"{}$_n$".format(dim) for dim in dims]
        text = r'$\approx$'.join([dims_n[i] for i in islice]) + r'$\approx 0$'
        ax.annotate(text, xy=(0.02, 0.98), xycoords='axes fraction', verticalalignment='top',
                    color='white')
    figname = f'x-z_slices_{node}_norm'
    save(figname)

### Corner

In [ ]:
cmap = pplt.Colormap('mono', left=0.065, right=0.9)
cmap

In [ ]:
for node in nodes:
    axes = mplt.corner(
        load_bunch(filenames['bunch'][node]), 
        labels=labels, kind='hist',
        mask_zero=True,
        cmap=cmap,
    )
    axes.format(xlabel_kw=dict(fontsize='large'), ylabel_kw=dict(fontsize='large'))
    save(f'corner_{node}')